In [58]:
# imports
import sqlite3
import operator
import re
import os

In [59]:
# create connection to sql db
sql_connect = sqlite3.connect('../data/lahmansbaseballdb.sqlite')
cursor = sql_connect.cursor()

# switches between retrosheet ID's and Lahman database ID's
def retro_to_lahman(retro_id):
    query = 'SELECT playerID FROM people WHERE retroID = "' + retro_id + '"'
    result = cursor.execute(query).fetchall()
    return result[0][0] if result else ''

In [268]:
# aggregates team data for an individual year
def get_team_stats(year):
    query = """
    SELECT teamIDretro, CAST(sum(W) as REAL) / (sum(W) + sum(L)),
    (sum(H) + sum(BB) + sum(HBP)) / CAST(sum(AB) + sum(BB) + sum(HBP) + sum(SF) as REAL),
    (sum(H) + sum("2B") + 2*sum("3B") + 3*sum(HR)) / CAST(sum(AB) as REAL),
    27 * sum(ER) / CAST(sum(IPouts) as REAL)
    FROM teams WHERE yearID = {y} GROUP BY teamIDretro;""".format(y = int(year))
    result = list(cursor.execute(query).fetchall())
    
    # convert list of tuples to dictionary
    team_stats = {}
    for t in result:
        team_stats[t[0]] = dict([('Win%', t[1]), ('OBP', t[2]), ('SLG', t[3]), ('ERA', t[4])])
    return team_stats

In [242]:
# aggregates all career data from all players up to, but not including, year
def get_career_stats(year):
    # batting
    batter_query = """
        SELECT playerID, (sum(H) + sum(BB) + sum(HBP) + sum(IBB))
        / CAST((sum(AB) + sum(BB) + sum(IBB) + sum(HBP) + sum(SF)) as REAL),
        (sum(H) + sum("2B") + 2*sum("3B") + 3*sum(HR)) / CAST(sum(AB) as REAL), 
        sum(HR), sum(BB), sum(RBI), sum(SO), sum(AB)
        FROM batting WHERE {y1} <= yearID AND yearID < {y2} GROUP BY playerID;""".format(y1 = int(year) - 30, y2 = int(year))
    # query goes thirty years back for long-time veterans
    result = list(cursor.execute(batter_query).fetchall())
    
    # convert list of tuples into dictionary
    career_stats = {}
    for p in result:
        career_stats[p[0]] = dict([('OBP', p[1]), ('SLG', p[2]), ('HR', p[3]),
                             ('BB', p[4]), ('RBI', p[5]), ('K', p[6]), ('AB', p[7])])
    # pitching
    pitcher_query = """
    SELECT playerID, sum(IPouts), sum(BB), sum(SO),
    CAST(sum(H) as REAL) / (sum(BFP) - sum(BB) - sum(HBP)), 27 * sum(ER) / CAST(sum(IPouts) as REAL), sum(HR)
    FROM pitching WHERE {y1} <= yearID AND yearID < {y2} GROUP BY playerID;
    """.format(y1 = int(year) - 30, y2 = int(year))
    result = list(cursor.execute(pitcher_query).fetchall())
    for p in result:
        career_stats[p[0] + "_PITCHER"] = dict([('Outs', p[1]), ('BB', p[2]), ('K', p[3]), ('BAOpp', p[4]),
                                               ('ERA', p[5]), ('HR', p[6])])
    return career_stats

In [243]:
# updates the season_stats dict based on an input series of plays, until the end date has been reached.
# the season_stats dict consists of pairs of a playerID key with a value of a dict containing that player's stats
def update_season_stats(season_stats, plays, end_date):
    home_pitcher = None
    away_pitcher = None
    for play in plays:
        fields = play.split(',')
        # reached the end date
        if play.startswith('id') and (re.search(".*" + end_date + ".", play) or int(play[-6:-2]) > int(end_date)):
            return season_stats # technically don't need return val, as season_stats param persists outside of function
        elif play.startswith(('start', 'sub')) and fields[5].strip() == "1":
            pitcher_name = fields[1] + "_PITCHER"
            if pitcher_name not in season_stats: # create empty dict if new pitcher
                season_stats[pitcher_name] = {}
            if fields[3].strip() == "1":
                home_pitcher = season_stats[pitcher_name]
            else:
                away_pitcher = season_stats[pitcher_name]
        elif play.startswith('play'):
            # create empty dict if new player
            if fields[3] not in season_stats:
                season_stats[fields[3]] = {}
            player = season_stats[fields[3]]
            
            # parse through specific play and increment specific values based on the play.
            # see https://www.retrosheet.org/datause.txt for reference
            batter_to_increment = []
            pitcher_to_increment = []
            if fields[6].startswith('S') and not fields[6].startswith('SB'): # single
                batter_to_increment += [('H', 1), ('TB', 1), ('AB', 1), ('PA', 1)]
                pitcher_to_increment += [('H', 1)]
            elif fields[6].startswith('D'): # double
                batter_to_increment += [('H', 1), ('TB', 2), ('AB', 1), ('PA', 1)]
                pitcher_to_increment += [('H', 1)]
            elif fields[6].startswith('T'): # triple
                batter_to_increment += [('H', 1), ('TB', 3), ('AB', 1), ('PA', 1)]
                pitcher_to_increment += [('H', 1)]
            elif fields[6].startswith('HR'): # home run
                batter_to_increment += [('H', 1), ('TB', 4), ('HR', 1), ('AB', 1), ('PA', 1)]
                pitcher_to_increment += [('H', 1), ('HR', 1)]
                if not fields[6].count('-H'): # sometimes HR's don't record base advancement
                    batter_to_increment += [('RBI', 1)]
                    pitcher_to_increment += [('ER', 1)]
            elif fields[6].startswith('W') or fields[6].startswith('IW'): # walk or intentional walk
                batter_to_increment += [('BB', 1), ('PA', 1)]
                if fields[6].startswith('W'): # don't count intentional walks
                    pitcher_to_increment += [('BB', 1)]
            elif fields[6].startswith('K'): # strikeout
                batter_to_increment += [('K', 1), ('AB', 1), ('PA', 1)]
                pitcher_to_increment += [('K', 1), ('Outs', 1)]
            elif 'SF' in fields[6]: # sac fly
                batter_to_increment += [('PA', 1)]
            elif not fields[6].startswith(('NP', 'HP', 'SB', 'CS')):
                batter_to_increment += [('AB', 1), ('PA', 1)]
                if "GDP" in fields[6]: # double play
                    pitcher_to_increment += [('Outs', 2)]
                else:
                    pitcher_to_increment += [('Outs', 1)]
            # for RBI's, each play dictates the advancement of baserunners, i.e. 2-H indicates going from second to home.
            # since each play doesn't explicitly track the RBI's, we can count the number of advancements to home instead
            batter_to_increment += [('RBI', fields[6].count('-H'))]
            pitcher_to_increment += [('ER', fields[6].count('-H'))]
            
            for key_value_pair in batter_to_increment:
                dict_add(player, key_value_pair)
            for key_value_pair in pitcher_to_increment:
                if fields[2].strip() == "0":
                    dict_add(home_pitcher, key_value_pair)
                else:
                    dict_add(away_pitcher, key_value_pair)
            
# one function for creation of dict entry and incrementing of dict entry
def dict_add(dictionary, key_value_pair):
    key, value = key_value_pair
    if key in dictionary:
        dictionary[key] += value
    else:
        dictionary[key] = value

In [329]:
# returns a given team's stats at the current date and year in the following format:
# first seven features: current season's Wins, Games Played, OBP, SLG, ERA, R/game, HR
# next four features: Win%, OBP, SLG, ERA of previous year
# next four features: Win%, OBP, SLG, ERA of two years previous
# next four features: Win%, OBP, SLG, ERA of three years previous
def team_stats(team, year, date, team_history):
    season = []
    games = open('../data/retrosheet_game_logs/GL' + str(year) + '.TXT').readlines()
    oldest_date = 0
    home_indices = [10, 9, 21, 22, 23, 24, 25, 30] # team score, opp score, AB, hits, doubles, triples, HRs, walks
    away_indices = [9, 10, 49, 50, 51, 52, 53, 58]
    wins, games_played, runs_for, runs_against, AB, hits, TB, HR, BB = 0, 0, 0, 0, 0, 0, 0, 0, 0
    for game in games:
        game = ''.join(x if i % 2 == 0 else x.replace(',', ':')
                       for i, x in enumerate(game.split('"'))) # remove commas within quotes
        game = game.replace('"', '')
        game_date = game[4:8]
        if int(game_date) >= int(date):
            break
        fields = game.split(',')
        if fields[3].strip() == team:
            i = away_indices
        elif fields[6].strip() == team:
            i = home_indices
        else:
            continue
        wins += 1 if int(fields[i[0]]) > int(fields[i[1]]) else 0
        games_played += 1
        runs_for += int(fields[i[0]])
        runs_against += int(fields[i[1]])
        AB += int(fields[i[2]])
        hits += int(fields[i[3]]) + int(fields[i[4]]) + int(fields[i[5]]) + int(fields[i[6]])
        TB += int(fields[i[3]]) + int(fields[i[4]]) + 2*int(fields[i[5]]) + 3*int(fields[i[6]])
        HR += int(fields[i[6]])
        BB += int(fields[i[7]])
    
    season = [wins, games_played, (hits + BB) / float(max(AB + BB, 1)), TB / float(max(AB, 1)),
              runs_against / float(max(games_played, 1)), runs_for / float(max(games_played, 1)), HR]
    
    # team history
    history = []
    one_year = team_history[0][team]
    history += [one_year["Win%"], one_year["OBP"], one_year["SLG"], one_year["ERA"]]
    two_year = team_history[1][team]
    history += [two_year["Win%"], two_year["OBP"], two_year["SLG"], two_year["ERA"]]
    three_year = team_history[2][team]
    history += [three_year["Win%"], three_year["OBP"], three_year["SLG"], three_year["ERA"]]
    
    return season + history

In [309]:
# returns a given batter's stats at the current date and year in the following format:
# first seven features: career OBP, SLG, HR, BB, RBI, K, AB
# next seven features: current season OBP, SLG, HR, BB, RBI, K, AB
def batter_stats(player, year, date, career_stats, season_stats):
    
    lahman_id = retro_to_lahman(player)
    
    # season stats
    if player not in season_stats or 'AB' not in season_stats[player]: # first player appearance
        season = [0.2, 0.3, 0, 0, 0, 0, 0] # approx replacement level OBP and SLG
    else:
        s = season_stats[player]
        season = [(s.get('H', 0) + s.get('BB', 0)) / float(s.get('PA', 1)), # OBP
                  s.get('TB', 0) / float(s.get('AB', 1)), # SLG
                  s.get('HR', 0), s.get('BB', 0), s.get('RBI', 0), s.get('K', 0), s.get('AB', 0)]
        
    # career stats
    if lahman_id not in career_stats or career_stats[lahman_id]['AB'] == 0: # rookie season
        career = [0, 0, 0, 0, 0, 0, 0]
    else:
        c = career_stats[lahman_id]
        career = [c['OBP'], c['SLG'], c['HR'], c['BB'], c['RBI'], c['K'], c['AB']]
            
    # incorporate current season into career stats
    
    # weighted average of OBP and SLG
    total_AB = season[6] + career[6]
    total_PA = total_AB + season[3] + career[3]
    if total_AB == 0 or total_PA == 0:
        career[0] = 0.2
        career_stats[1] = 0.3
    else:
        career[0] = (season[0] * (season[6] + season[3]) + career[0] * (career[6] + career[3])) / total_PA
        career[1] = (season[1] * season[6] + career[1] * career[6]) / total_AB
    # add all of the counting stats
    for i in [2, 3, 4, 5, 6]:
        career[i] += season[i]
    
    return career + season

In [306]:
# returns a given pitcher's stats at the current date and year in the following format:
# first six features: career Outs, BB, K, BAOpp, ERA, HR
# next six features: current season Outs, BB, K, BAOpp, ERA, HR
def pitcher_stats(player, year, date, career_stats, season_stats):
    lahman_id = retro_to_lahman(player) + "_PITCHER"
    player += "_PITCHER"
    if player not in season_stats or season_stats[player]['Outs'] == 0: # first pitching appearance
        season = [0.2, 0.3, 0, 0, 0.3, 6, 0] # approx replacement level BAOpp and ERA
    else:
        s = season_stats[player]
        season = [s.get('Outs', 0), s.get('BB', 0), s.get('K', 0), float(s.get('H', 0)) / (s.get('H', 0) + s.get('Outs', 1)),
                  27.0 * s.get('ER', 0) / s.get('Outs', 1), s.get('HR', 0)]
        
    # career stats
    if lahman_id not in career_stats or career_stats[lahman_id]['Outs'] == 0: # rookie season
        career = [0, 0, 0, 0, 0, 0]
    else:
        c = career_stats[lahman_id]
        career = [c['Outs'], c['BB'], c['K'], c['BAOpp'], c['ERA'], c['HR']]
        
    # incorporate current season into career stats
    
    # weighted average of BAOpp and ERA
    total_outs = career[0] + season[0]
    if total_outs == 0:
        career[3] = 0.3
        career[4] = 6
    elif player in season_stats:
        total_hits = season_stats[player].get('H', 0) + career[0] * career[3] / (1 - career[3])
        total_runs = season_stats[player].get('ER', 0) + career[0] * career[4] / 27.0
        career[3] = float(total_hits) / (total_hits + total_outs)
        career[4] = 27.0 * total_runs / total_outs
    # add in all of the counting stats
    for i in [0, 1, 2, 5]:
        career[i] += season[i]
    
    return career + season

In [304]:
home_batters_idx = [132, 135, 138, 141, 144, 147, 150, 153, 156]
away_batters_idx = [105, 108, 111, 114, 117, 120, 123, 126, 129]

# generates the features and label of a given game
def example(game, career_stats, season_stats, team_history):
    fields = game.split(',')
    year = fields[0][0:4]
    date = fields[0][4:8]
    home_team = fields[6]
    away_team = fields[3]
    home_pitcher = fields[103]
    away_pitcher = fields[101]
    label = 1 if int(fields[10]) - int(fields[9]) > 0 else 0 # 1 if home team won
    
    # add features
    features = []
    features += team_stats(home_team, year, date, team_history)
    features += team_stats(away_team, year, date, team_history)
    for idx in home_batters_idx:
        features += batter_stats(fields[idx], year, date, career_stats, season_stats)
    features += pitcher_stats(home_pitcher, year, date, career_stats, season_stats)
    for idx in away_batters_idx:
        features += batter_stats(fields[idx], year, date, career_stats, season_stats)
    features += pitcher_stats(away_pitcher, year, date, career_stats, season_stats)
    
    return features, label

In [328]:
# creates examples for a given year in time
def season(year, verbose=True):
    features, labels = [], []
    games = open('../data/retrosheet_game_logs/GL' + str(year) + '.TXT').readlines()
    oldest_date = 0
    career_stats = get_career_stats(year) # career stats don't change
    season_stats = {} # season stats change as the season progresses
    team_history = [get_team_stats(year - 1), get_team_stats(year - 2), get_team_stats(year - 3)]
    print_counter = 1000
    for game in games:
        game = ''.join(x if i % 2 == 0 else x.replace(',', ':')
                       for i, x in enumerate(game.split('"'))) # remove commas within quotes
        game = game.replace('"', '')
        date = game[4:8]
        
        print_counter += 1
        if print_counter > 350 and verbose:
            print_counter = 0
            print('Evaluating game ' + date + " " +  str(year))
        if int(date) > oldest_date: # update stats to new date
            oldest_date = int(date)
            season_stats = get_season_stats(year, date)
        feature, label = example(game, career_stats, season_stats, team_history)
        features.append(feature)
        labels.append(label)
    return features, labels
    
# gets season stats of year until end_date
def get_season_stats(year, end_date):
    season_stats = {}
    for file in os.listdir('../data/retrosheet_event_logs'):
        if re.search(str(year) + '.*EV.', file):
            update_season_stats(season_stats, open('../data/retrosheet_event_logs/' + file), end_date)
    return season_stats

In [330]:
for i in range(1987, 2020):
    features, labels = season(i)
    
    print("Saving year " + str(i) + "...")
    # save to files
    with open('../data/features.txt', 'a') as f:
        for feature in features:
            f.write("%s\n" % feature)

    with open('../data/labels.txt', 'a') as f:
        for label in labels:
            f.write("%s\n" % label)

Evaluating game 0408 1981
Evaluating game 0511 1981
Evaluating game 0610 1981
Evaluating game 0906 1981
Evaluating game 0405 1982
Evaluating game 0509 1982
Evaluating game 0609 1982
Evaluating game 0708 1982
Evaluating game 0807 1982
Evaluating game 0905 1982
Evaluating game 1003 1982
Evaluating game 0404 1983
Evaluating game 0510 1983
Evaluating game 0610 1983
Evaluating game 0710 1983
Evaluating game 0806 1983
Evaluating game 0903 1983
Evaluating game 1002 1983
Evaluating game 0402 1984
Evaluating game 0506 1984
Evaluating game 0607 1984
Evaluating game 0705 1984
Evaluating game 0804 1984
Evaluating game 0901 1984
Evaluating game 0408 1985
Evaluating game 0511 1985
Evaluating game 0610 1985
Evaluating game 0709 1985
Evaluating game 0811 1985
Evaluating game 0909 1985
Evaluating game 0407 1986
Evaluating game 0510 1986
Evaluating game 0608 1986
Evaluating game 0707 1986
Evaluating game 0808 1986
Evaluating game 0906 1986
Evaluating game 0406 1987


KeyboardInterrupt: 